

**Problem Statement:**

Goal: Create a unified hybrid framework that adapts to the commodity type and automatically selects or combines the best models accordingly.

We loaded historical data from 2003-April 2025 alonmg with new realtime data to predict price with pre trained model.

## Step 8 Processing Real Time Data with Pre Trained Models

##### Sourcing Key Libraries


In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np 
import os 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import statsmodels.formula.api as smf
# from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima.model import ARIMA
#from pmdarima.arima import auto_arima
import pmdarima as pm
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestRegressor
import os
import itertools
import requests
import yfinance as yf
from scipy.stats import randint
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential
#from arch import arch_model
from transformers import pipeline
#from xgboost import XGBRegressor
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

##### Define constant Variables

In [2]:
# constant values
# Your API Key from FRED
api_key = 'a27b910873da479a5561ea08035a6c79'
# API Key for Alpha Vantage (Replace with your actual key)
API_KEY_alpha = "3J4EGZCB0D7UT9WG"
# genertae your APi key from here: https://www.alphavantage.co/

#update the below code for any new precious Metal

data_ticker_map = {
            'GOLD' : 'GC=F',
            'SILVER': 'SI=F',
            'CRUDE_OIL': 'CL=F'}

data_file_mapping = {
            'GOLD' : '../Data/Historical_Gold_data_April_2020.csv',
           'SILVER': '../Data/Historical_Silver_data_April.csv',
           'CRUDE_OIL': '../Data/Historical_Crude_oil_data_April.csv',}

Live_Sentiments_file_name = 'real-Live_Sentiments.csv'
Live_Sentiments_file_dir = '../Sentiments'
Historical_Sentiments_file_name = 'Historical_Sentiments.csv'
Historical_Sentiments_file_dir = '../Sentiments'

best_models_dir = {'GOLD': '../Models/GOLD_MODELS', 'SILVER' : '../Models/SILVER_MODELS', 'CRUDE_OIL': '../Models/CRUDE_OIL_MODELS'}
hybrid_models_dir = {'GOLD': '../Models/GOLD_HYBRID_MODELS', 'SILVER' : '../Models/SILVER_HYBRID_MODELS', 'CRUDE_OIL': '../Models/CRUDE_OIL_HYBRID_MODELS'}
hybrid_meta_models_dir = {'GOLD': '../Models/GOLD_HYBRID_MODELS/Meta', 'SILVER' : '../Models/SILVER_HYBRID_MODELS/Meta', 'CRUDE_OIL': '../Models/CRUDE_OIL_HYBRID_MODELS/Meta'}

### Enter Commodity Name for which you want to predict Price EX: GOLD, SILVER, CRUDE OIL etc

In [3]:
commodity = input('Enter the Commodity Name')
filename =  data_file_mapping.get(commodity)
############# Reading Data Sources #################
print(f'Loading from Historical file id Data available in file : {filename}')
ticker = data_ticker_map.get(commodity)
print(f'Ticker code for {commodity} : {ticker}')

Enter the Commodity Name GOLD


Loading from Historical file id Data available in file : ../../Data/Historical_Gold_data_April_2020_2.csv
Ticker code for GOLD : GC=F


In [4]:
d = '30' #input('Enter the number of Days/Hours you eant to predict from real time 30,15,90')
days = int(d)
realtimedays = d+'d'
realtimedays

'30d'

## Data Sourcing

### Historical Data Sourcing

##### Yahoo finance is not reliable and having issues, download data from 2003-01-01 - 2025-03-15 via yf api.

##### Define function to download Historical data 

In [5]:
# Function to download historical price data
def download_price_data(ticker, start='2020-01-01', end='2025-04-28'):
    # Download the data
    data = yf.download(ticker, start=start, end=end)
    
    # Reset index to turn 'Date' into a regular column
    data.reset_index(inplace=True)
    
    # Flatten the columns if there are multiple levels
    data.columns = data.columns.get_level_values(0) if isinstance(data.columns, pd.MultiIndex) else data.columns
    
    # Select necessary columns only
    data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    return data

# Load or download data for each commodity
def load_or_download_data(ticker, filename):
    if os.path.exists(filename):
        # Read the CSV, treating the first row as the header
        # data = pd.read_csv(filename, header=0)
        data = pd.read_csv(filename, header=0)
        # Convert 'Date' column to datetime format, and handle parsing errors
        data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
        
        # Remove any rows where 'Date' could not be parsed and resulted in NaT
        data = data.dropna(subset=['Date'])
    else:
        # Download the data
        data = download_price_data(ticker)
        
        # Save to CSV with a single header row
        data.to_csv(filename, index=False)
    
    # Ensure the 'Close' column is numeric
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    
    return data


In [6]:
############# Reading Data Sources #################
print(f'Loading from Historical file id Data available in file : {filename}')
ticker = data_ticker_map.get(commodity)
print(f'Ticker code for {commodity} : {ticker}')
commodity_data = load_or_download_data(ticker, filename)

Loading from Historical file id Data available in file : ../../Data/Historical_Gold_data_April_2020_2.csv
Ticker code for GOLD : GC=F


### Fetch Realtime Data

In [7]:
# Function to fetch real-time data
def fetch_real_time_data(ticker, period, interval):
    data = yf.download(ticker, period=period, interval=interval)
    data.columns = data.columns.get_level_values(0) if isinstance(data.columns, pd.MultiIndex) else data.columns
    data.columns.name = None
    data.index = pd.to_datetime(data.index).date
    # Select and standardize the required columns
    data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    return data

###  Realtime Data process

In [8]:

real_time_data = fetch_real_time_data(ticker, period=realtimedays, interval='1d')
# Convert index to Date column
real_time_data['Date'] = real_time_data.index
# Reset index
real_time_data.reset_index(drop=True, inplace=True)
# Ensure 'Date' is datetime without timezone
real_time_data['Date'] = pd.to_datetime(real_time_data['Date']).dt.normalize()
# Reorder columns
real_time_data = real_time_data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]

# Print output
print(real_time_data.head())
# ######## Combine Real TIme data with Historical data
commodity_data = pd.concat([commodity_data,real_time_data])
commodity_data = commodity_data.drop_duplicates(subset='Date', keep='first')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

        Date         Open         High          Low        Close  Volume
0 2025-03-28  3069.699951  3094.899902  3066.800049  3086.500000   31206
1 2025-03-31  3091.000000  3132.500000  3086.000000  3122.800049    3438
2 2025-04-01  3129.699951  3149.500000  3104.000000  3118.899902    1721
3 2025-04-02  3120.699951  3168.600098  3117.399902  3139.899902    5946
4 2025-04-03  3150.000000  3166.899902  3052.000000  3097.000000    5516


### Sourcing Market data

####  GDP data

##### Define function which will convert Yearly GDP data to daily GDP data

In [9]:
def expand_gdp_to_daily(gdp_df):
    daily_gdp = []
    for index, row in gdp_df.iterrows():
        month_start = row['Date']
        value = row['Value']
        month_days = pd.date_range(start=month_start, end=month_start + pd.offsets.MonthEnd(0), freq='D')
        for day in month_days:
            daily_gdp.append({'Date': day, 'Value': value})
    return pd.DataFrame(daily_gdp)


def preprocess_gdp_data(gdp_data):
    gdp_data['Date'] = pd.to_datetime(gdp_data['Date'])
    return expand_gdp_to_daily(gdp_data)


def fetch_world_bank_data(indicator, country='USA', start_year='2003', end_year='2025'):
    url = f'http://api.worldbank.org/v2/country/{country}/indicator/{indicator}?date={start_year}:{end_year}&format=json'
    try:
        response = requests.get(url)
        data = response.json()
        if len(data) < 2 or 'message' in data[1]:
            print("No data found for the specified parameters.")
            return None
        df = pd.json_normalize(data[1])
        df['date'] = pd.to_datetime(df['date'])
        df = df[(df['date'] >= f'{start_year}-01-01') & (df['date'] <= f'{end_year}-12-31')]
        df = df[['date', 'value']]
        df.columns = ['Date', 'Value']
        return df
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None


In [10]:
############### sourcing world GDP data #################################################
gdp_data = fetch_world_bank_data('NY.GDP.MKTP.CD', country='USA', start_year='2003', end_year='2024')
daily_gdp_data = preprocess_gdp_data(gdp_data)
daily_gdp_data.rename(columns={'Value':'GDP'},inplace=True)
daily_gdp_data

Date               GDP
0   2024-01-01               NaT
1   2024-01-02               NaT
2   2024-01-03               NaT
3   2024-01-04               NaT
4   2024-01-05               NaT
..         ...               ...
677 2003-01-27  11456450000000.0
678 2003-01-28  11456450000000.0
679 2003-01-29  11456450000000.0
680 2003-01-30  11456450000000.0
681 2003-01-31  11456450000000.0

[682 rows x 2 columns]

####  Interest rate data

In [11]:

############## Sourcing Interest rate data ################################
from fredapi import Fred

# Your API Key from FRED
api_key = 'a27b910873da479a5561ea08035a6c79'

# Initialize the FRED API
fred = Fred(api_key=api_key)

# Fetch the Federal Funds Rate data (you can change this to any other indicator)
# Federal Funds Rate is often identified by the FRED ID 'FEDFUNDS'
interest_rate_data = fred.get_series('FEDFUNDS' ,start_date='2003-01-01')

# Convert the data into a DataFrame for better handling
interest_rate_df = pd.DataFrame(interest_rate_data)
interest_rate_df.columns = ['Interest_rates']

# Reset the index to have 'Date' as a column
interest_rate_df.reset_index(inplace=True)

import pandas as pd

# Assuming `interest_rate_df` already exists with 'Date' and 'Interest_rates' columns

# Convert 'Date' column to datetime format if it’s not already
interest_rate_df['Date'] = pd.to_datetime(interest_rate_df['index'])
interest_rate_df.drop(columns=['index'], inplace=True)  # Drop the old index column if needed

# Create a date range for every day from the start to the end of the interest rate data
start_date = interest_rate_df['Date'].min()
end_date = interest_rate_df['Date'].max()
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Create a new DataFrame with daily dates as a column
daily_interest_rate_df = pd.DataFrame({'date': date_range})

# Add a 'month' and 'year' column to `interest_rate_df` for easy merging
interest_rate_df['year'] = interest_rate_df['Date'].dt.year
interest_rate_df['month'] = interest_rate_df['Date'].dt.month

# Merge monthly data with daily data on matching 'year' and 'month'
daily_interest_rate_df['year'] = daily_interest_rate_df['date'].dt.year
daily_interest_rate_df['month'] = daily_interest_rate_df['date'].dt.month

# Perform a left join on 'year' and 'month' columns
daily_interest_rate_df = daily_interest_rate_df.merge(
    interest_rate_df[['year', 'month', 'Interest_rates']],
    on=['year', 'month'],
    how='left'
)

# Drop the extra columns, keep only 'date' and 'Interest_rates'
daily_interest_rate_data = daily_interest_rate_df[['date', 'Interest_rates']]
daily_interest_rate_data.rename(columns={'date':'Date'},inplace=True)
daily_interest_rate_data


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_29016\1493415932.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_interest_rate_data.rename(columns={'date':'Date'},inplace=True)


Date  Interest_rates
0     1954-07-01            0.80
1     1954-07-02            0.80
2     1954-07-03            0.80
3     1954-07-04            0.80
4     1954-07-05            0.80
...          ...             ...
25838 2025-03-28            4.33
25839 2025-03-29            4.33
25840 2025-03-30            4.33
25841 2025-03-31            4.33
25842 2025-04-01            4.33

[25843 rows x 2 columns]

####  GDP to Debt data

In [12]:
################## GDP to Debt DATA ###############################
import wbdata
import pandas as pd
import datetime

# Set the start and end dates for the data
start_date = datetime.datetime(2000, 1, 1)
end_date = datetime.datetime(2023, 1, 1)

# List of indicators to retrieve (Government Debt to GDP ratio)
indicators = {'GC.DOD.TOTL.GD.ZS': 'Government Debt to GDP'}

# Fetch data from the World Bank API for the USA
government_debt_data = wbdata.get_dataframe(indicators, country='USA')

# Fill missing values in `Government Debt to GDP` with 0
government_debt_data['Government Debt to GDP'] = government_debt_data['Government Debt to GDP'].fillna(0)

# Reset index to make `date` a regular column and convert it to datetime
government_debt_data.reset_index(inplace=True)
government_debt_data['date'] = pd.to_datetime(government_debt_data['date'])

# Add a 'year' column to `government_debt_data` to help with mapping
government_debt_data['year'] = government_debt_data['date'].dt.year

# Create a dictionary for quick lookup of Debt to GDP per year
debt_to_gdp_dict = government_debt_data.set_index('year')['Government Debt to GDP'].to_dict()

# Create a date range for every day from the start to the end date
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Create a new DataFrame with daily dates as a column
daily_debt_data = pd.DataFrame({'date': date_range})

# Map each date in `daily_debt_data` to the corresponding Debt to GDP value for that year
daily_debt_data['DebtToGDP'] = daily_debt_data['date'].dt.year.map(debt_to_gdp_dict)
daily_debt_data.rename(columns={'date':'Date'},inplace=True)

daily_debt_data

Date   DebtToGDP
0    2000-01-01   33.267642
1    2000-01-02   33.267642
2    2000-01-03   33.267642
3    2000-01-04   33.267642
4    2000-01-05   33.267642
...         ...         ...
8397 2022-12-28  110.385332
8398 2022-12-29  110.385332
8399 2022-12-30  110.385332
8400 2022-12-31  110.385332
8401 2023-01-01    0.000000

[8402 rows x 2 columns]

####  Inflation data

In [13]:
############## inflation data #################
# Get historical CPI data (Consumer Price Index) to calculate inflation
cpi_data = fred.get_series('CPIAUCSL', start_date='2003-01-01')



# Convert the data into a DataFrame for better handling
Inflation_data = pd.DataFrame(interest_rate_data)
Inflation_data.columns = ['Inflation_rate']

# Reset the index to have 'Date' as a column
Inflation_data.reset_index(inplace=True)
Inflation_data.rename(columns={'index':'Date'},inplace=True)
# Print the first few rows
Inflation_data

Date  Inflation_rate
0   1954-07-01            0.80
1   1954-08-01            1.22
2   1954-09-01            1.07
3   1954-10-01            0.85
4   1954-11-01            0.83
..         ...             ...
845 2024-12-01            4.48
846 2025-01-01            4.33
847 2025-02-01            4.33
848 2025-03-01            4.33
849 2025-04-01            4.33

[850 rows x 2 columns]

####  ETF data

In [14]:
################ Gold ETF data ###########################################
import requests
import pandas as pd

# Fetch ETF data from FRED API
def fetch_fred_etf_data(series_id, api_key, start_date='2003-01-01', end_date='2024-11-01'):
    url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json&observation_start={start_date}&observation_end={end_date}'
    response = requests.get(url)
    data = response.json()
    
    # Print the entire JSON response for inspection
    print("Full JSON response:", data)
    
    # Check for errors in the JSON response
    if "observations" not in data:
        print("Error: 'observations' key not found in the response. Check the series ID or API request.")
        return None
    
    # Parse JSON data into a DataFrame
    etf_df = pd.DataFrame(data['observations'])
    etf_df['Date'] = pd.to_datetime(etf_df['date'])
    etf_df['Date'] = etf_df['Date'].dt.tz_localize(None).dt.date 
    etf_df['value'] = pd.to_numeric(etf_df['value'], errors='coerce')
    etf_df.set_index('Date', inplace=True)
    
    return etf_df

# Example usage
api_key = 'a27b910873da479a5561ea08035a6c79'
series_id = 'INTDSRUSM193N'  # Replace with a valid series ID from FRED
etf_data = fetch_fred_etf_data(series_id, api_key)


etf_data.rename(columns={'value':'ETF_Value'},inplace=True)
# Only print if data retrieval was successful
if etf_data is not None:
    print(etf_data.head())

Full JSON response: {'realtime_start': '2025-04-18', 'realtime_end': '2025-04-18', 'observation_start': '2003-01-01', 'observation_end': '2024-11-01', 'units': 'lin', 'output_type': 1, 'file_type': 'json', 'order_by': 'observation_date', 'sort_order': 'asc', 'count': 224, 'offset': 0, 'limit': 100000, 'observations': [{'realtime_start': '2025-04-18', 'realtime_end': '2025-04-18', 'date': '2003-01-01', 'value': '2.25'}, {'realtime_start': '2025-04-18', 'realtime_end': '2025-04-18', 'date': '2003-02-01', 'value': '2.25'}, {'realtime_start': '2025-04-18', 'realtime_end': '2025-04-18', 'date': '2003-03-01', 'value': '2.25'}, {'realtime_start': '2025-04-18', 'realtime_end': '2025-04-18', 'date': '2003-04-01', 'value': '2.25'}, {'realtime_start': '2025-04-18', 'realtime_end': '2025-04-18', 'date': '2003-05-01', 'value': '2.25'}, {'realtime_start': '2025-04-18', 'realtime_end': '2025-04-18', 'date': '2003-06-01', 'value': '2'}, {'realtime_start': '2025-04-18', 'realtime_end': '2025-04-18', 'd

## Sourcing Sentiment Data and Apply BERT

#### Sourcing Live Sentiments

In [15]:
import requests
import pandas as pd
from transformers import pipeline
from datetime import datetime, timedelta

# Set up NewsAPI
API_KEY = "250e051931514de6b0e5120412c4e3ad"
NEWS_API_URL = "https://newsapi.org/v2/everything"

# Fetch news using query
def fetch_news(query, language="en", page_size=100):
    params = {
        "q": query,
        "language": language,
        "apiKey": API_KEY,
        "pageSize": page_size,
        "sortBy": "publishedAt",
    }
    response = requests.get(NEWS_API_URL, params=params)
    data = response.json()
    
    if data.get("status") == "ok":
        articles = data.get("articles", [])
        return [(article["publishedAt"], article["title"]) for article in articles]
    else:
        print("Error fetching news:", data)
        return []

# Load FinBERT pipeline
sentiment_pipeline = pipeline("text-classification", model="ProsusAI/finbert")

def analyze_sentiment(texts):
    sentiments = sentiment_pipeline(texts, truncation=True)
    return [sent["label"] for sent in sentiments]

# Combined sentiment fetcher for crude oil-related queries
def get_gold_sentiment():
    queries =  [
    "gold price",
    "gold futures",
    "gold market",
    "gold trading",
    "gold commodity",
    "gold ETF",
    "precious metals",
    "gold demand",
    "gold mining"
]
    all_articles = []

    for query in queries:
        articles = fetch_news(query)
        all_articles.extend(articles)

    if not all_articles:
        return pd.DataFrame()

    df = pd.DataFrame(all_articles, columns=["Date", "Headline"])
    df["Sentiment"] = analyze_sentiment(df["Headline"].tolist())
    df.to_csv("gold_sentiment_data.csv", index=False)
    return df


#### Apply Sentiment analysis on Live data and save them in disk

In [16]:
# Save the new data
import os
Live_Sentiments_file = os.path.join(Live_Sentiments_file_dir, commodity.upper(), Live_Sentiments_file_name)
if os.path.exists(Live_Sentiments_file):
    # sentiment_df = get_gold_sentiment()
    # sentiment_df.to_csv(Live_Sentiments_file, index=False)
    sentiment_df =  pd.read_csv(Live_Sentiments_file)  # File does not exist
else:
    # Fetch sentiment data
    sentiment_df = get_gold_sentiment()
    sentiment_df.to_csv(Live_Sentiments_file, index=False)
print(sentiment_df.head())
sentiment_df['Sentiment_score'] = sentiment_df['Sentiment'].apply(lambda x: 1 if x == 'positive' else -1 if x == 'negative' else 0)
print(sentiment_df.head())

# Add sentiment data back to the news_df
sentiment_df[['Date', 'Headline', 'Sentiment', 'Sentiment_score']]
live_sentiments_df = sentiment_df[['Date', 'Headline', 'Sentiment', 'Sentiment_score']]
# # Print out the resulting dataframe
# print(news_df[['Date', 'Title', 'Sentiment']].head())

# Convert Date column to datetime and add a range of days around each event
live_sentiments_df['Date'] = pd.to_datetime(live_sentiments_df['Date'])



live_sentiments_df['Date'] = pd.to_datetime(live_sentiments_df['Date'], errors='coerce')
live_sentiments_df['Date'] = live_sentiments_df['Date'].dt.tz_localize(None).dt.date 
live_sentiments_df.head()

                   Date                                           Headline  \
0  2025-05-01T16:22:23Z  Whatever happens with Trump’s tariffs, investo...   
1  2025-05-01T16:19:25Z  Iconic Portrait of Buzz Aldrin on the Moon Sel...   
2  2025-05-01T16:15:51Z                         Today’s News in a Nutshell   
3  2025-05-01T16:06:08Z  $80 Switch 2 and now Xbox games aren't a big s...   
4  2025-05-01T16:02:00Z  Norm Macdonald’s ‘Dirty Work’ Gets Even Dirtie...   

  Sentiment  
0   neutral  
1   neutral  
2   neutral  
3   neutral  
4  negative  
                   Date                                           Headline  \
0  2025-05-01T16:22:23Z  Whatever happens with Trump’s tariffs, investo...   
1  2025-05-01T16:19:25Z  Iconic Portrait of Buzz Aldrin on the Moon Sel...   
2  2025-05-01T16:15:51Z                         Today’s News in a Nutshell   
3  2025-05-01T16:06:08Z  $80 Switch 2 and now Xbox games aren't a big s...   
4  2025-05-01T16:02:00Z  Norm Macdonald’s ‘Dirty Work’ Ge

Date                                           Headline Sentiment  \
0  2025-05-01  Whatever happens with Trump’s tariffs, investo...   neutral   
1  2025-05-01  Iconic Portrait of Buzz Aldrin on the Moon Sel...   neutral   
2  2025-05-01                         Today’s News in a Nutshell   neutral   
3  2025-05-01  $80 Switch 2 and now Xbox games aren't a big s...   neutral   
4  2025-05-01  Norm Macdonald’s ‘Dirty Work’ Gets Even Dirtie...  negative   

   Sentiment_score  
0                0  
1                0  
2                0  
3                0  
4               -1

#### Load Historical Sentiment data

In [17]:
Historical_Sentiments_file = os.path.join(Historical_Sentiments_file_dir, commodity.upper(), Historical_Sentiments_file_name)
#Detect Invalid Dates
def is_invalid_date(date_str):
    try:
        pd.to_datetime(date_str, format="%d-%m-%Y", errors="raise")
        return False  # Valid date
    except:
        return True   # Invalid date



if os.path.exists(Historical_Sentiments_file) and commodity.upper()=="GOLD":
        historical_news_df =  pd.read_csv(Historical_Sentiments_file)  # File does not exist
        print(historical_news_df)
elif commodity.upper()=="GOLD" :
    # Fetch sentiment data
    historical_sentiment_data = pd.read_csv("C:/Users/Lenovo/OneDrive/Desktop/War/code/data/gold_historical_sentiments.csv", usecols=["Dates","Price Sentiment", "News"])
    historical_sentiment_data = historical_sentiment_data[~historical_sentiment_data["Dates"].astype(str).apply(is_invalid_date)]
    
    
    # Add sentiment data back to the news_df
    # # Print out the resulting dataframe
    # print(news_df[['Date', 'Title', 'Sentiment']].head())
    historical_news_df = historical_sentiment_data[['Dates', 'News', 'Price Sentiment', 'Sentiment_score']]
    historical_news_df.rename(columns = {'News':'Headline', 'Dates':'Date', 'Price Sentiment':'Sentiment'}, inplace = True)
    
    # Convert Date column to datetime and add a range of days around each event
    historical_news_df['Date'] = pd.to_datetime(historical_news_df['Date'])
    historical_news_df['Date'] = pd.to_datetime(historical_news_df['Date'], errors='coerce')
    historical_news_df['Date'] = historical_news_df['Date'].dt.tz_localize(None).dt.date 

    historical_news_df = historical_news_df[['Date','Headline']]
    historical_news_df["Sentiment_score"] = analyze_sentiment(historical_news_df["Headline"].tolist(), batch_size=16)
    historical_news_df['Sentiment_score'] = historical_news_df['Sentiment_score'].apply(lambda x: 1 if x == 'positive' else -1 if x == 'negative' else 0)
    historical_news_df.to_csv(Historical_Sentiments_file, index=False)
else:
    historical_news_df=live_sentiments_df




             Date                                           Headline  \
0      2016-01-28  april gold down 20 cents to settle at $1,116.1...   
1      2017-09-13          gold suffers third straight daily decline   
2      2016-07-26     Gold futures edge up after two-session decline   
3      2018-02-28  dent research : is gold's day in the sun comin...   
4      2017-09-06  Gold snaps three-day rally as Trump, lawmakers...   
...           ...                                                ...   
10400  2013-01-07       gold seen falling from 3-week high this week   
10401  2018-09-27  dominic frisby : now looks like a good time to...   
10402  2017-03-03  Gold heading for worst week since November on ...   
10403  2008-06-11  august gold up $7.60 at $878.80 an ounce on nymex   
10404  2007-10-01    december gold down $1 at $749 an ounce on nymex   

      BERT_Sentiment  Sentiment_score  
0           negative               -1  
1           negative               -1  
2           pos

## Step 2 : Data Preprocessing and Cleaning

### 2.1 Null Value Treatment

In [18]:
for data in [commodity_data]:
    data.dropna(subset=['Date', 'Close'], inplace=True)

### 2.2 Converting Values to Datetime format

In [19]:
# COnvert into date format
# Historical Data
# Convert Date column in gold_data to remove timezone
commodity_data['Date'] = commodity_data['Date'].dt.tz_localize(None).dt.date 

# Market Economic Data
daily_gdp_data['Date'] = daily_gdp_data['Date'].dt.tz_localize(None).dt.date 
interest_rate_df['Date'] = interest_rate_df['Date'].dt.tz_localize(None).dt.date 
Inflation_data['Date'] = Inflation_data['Date'].dt.tz_localize(None).dt.date 
daily_debt_data['Date'] = daily_debt_data['Date'].dt.tz_localize(None).dt.date 

### 2.3 Merging all source data to final data frame

In [20]:
# Merge all dataframes on 'Date' with 'gold_data' as the base (left join)
merged_data = commodity_data.merge(daily_gdp_data, on='Date', how='left') \
                       .merge(interest_rate_df, on='Date', how='left') \
                       .merge(Inflation_data, on='Date', how='left') \
                       .merge(daily_debt_data, on='Date', how='left') \
                        .merge(etf_data[['ETF_Value']], on='Date', how='left')

merged_data.drop(columns=['year','month'],inplace=True)

In [21]:
merged_data.info()
# Print summary of missing values after imputation
print(merged_data.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1342 entries, 0 to 1341
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            1342 non-null   object 
 1   Open            1342 non-null   float64
 2   High            1342 non-null   float64
 3   Low             1342 non-null   float64
 4   Close           1342 non-null   float64
 5   Volume          1342 non-null   int64  
 6   GDP             80 non-null     object 
 7   Interest_rates  42 non-null     float64
 8   Inflation_rate  42 non-null     float64
 9   DebtToGDP       756 non-null    float64
 10  ETF_Value       12 non-null     float64
dtypes: float64(8), int64(1), object(2)
memory usage: 115.5+ KB
Date                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
GDP               1262
Interest_rates    1300
Inflation_rate    1300
DebtToGDP          586
ETF_Value     

### 2.4 Processing NULL values from merged data

In [22]:

# Convert numeric columns from object type to float (if needed)
numeric_cols = ["Open", "High", "Low", "Close", "Volume"]
merged_data[numeric_cols] = merged_data[numeric_cols].apply(pd.to_numeric, errors="coerce")

# Handle Financial Market Data (Time-series features)
merged_data[numeric_cols] = merged_data[numeric_cols].fillna(method="ffill")  # Forward Fill (Recommended)

#  Handle Volume (Typically Skewed, Use Median)
merged_data["Volume"].fillna(merged_data["Volume"].median(), inplace=True)

# 🔹 Handle Economic Indicators (Use Mean or Interpolation)
econ_cols = ["GDP", "DebtToGDP", "Interest_rates", "Inflation_rate", "ETF_Value"]
for col in econ_cols:
    merged_data[col].fillna(merged_data[col].mean(), inplace=True)  # Mean Imputation
    merged_data[col].fillna(0, inplace=True)  # Mean Imputation


# Print summary of missing values after imputation
print(merged_data.isnull().sum())

commodity_data=merged_data

Date              0
Open              0
High              0
Low               0
Close             0
Volume            0
GDP               0
Interest_rates    0
Inflation_rate    0
DebtToGDP         0
ETF_Value         0
dtype: int64


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_29016\3625551758.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[numeric_cols] = merged_data[numeric_cols].fillna(method="ffill")  # Forward Fill (Recommended)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_29016\3625551758.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data["Volume"].fillna(merged_data["Volume"].median(), inplace=True)
C:\Users\Lenovo\AppData\Local\

### Outlier Treatment

In [23]:

# Function to identify and remove outliers using IQR
def identify_outliers(df):
    # Ensure 'Close' is numeric
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')  # Convert to float, set errors to NaN
    df = df.dropna(subset=['Close'])  # Drop rows where 'Close' is NaN after conversion

    Q1 = df['Close'].quantile(0.25)
    Q3 = df['Close'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    ndf = df[(df['Close'] <= lower_bound) | (df['Close'] >= upper_bound)]
    return ndf

# Apply outlier detection to the Gold dataset
commodity_data['Close'] = pd.to_numeric(commodity_data['Close'], errors='coerce')  # Ensure 'Close' is numeric
outlier = identify_outliers(commodity_data)
outlier

Date         Open         High          Low        Close  Volume  \
1081  2024-04-19  2379.300049  2399.800049  2377.300049  2398.399902     160   
1101  2024-05-17  2380.699951  2415.800049  2380.699951  2412.199951     213   
1102  2024-05-20  2415.800049  2435.800049  2409.699951  2433.899902      40   
1103  2024-05-21  2429.500000  2429.500000  2421.000000  2421.699951       6   
1137  2024-07-11  2378.699951  2416.699951  2378.699951  2415.000000      72   
...          ...          ...          ...          ...          ...     ...   
1337  2025-04-28  3284.500000  3332.500000  3284.500000  3332.500000    1300   
1338  2025-04-29  3340.000000  3340.000000  3299.899902  3318.800049    1278   
1339  2025-04-30  3318.699951  3318.699951  3268.000000  3305.000000     207   
1340  2025-05-01  3272.899902  3275.000000  3198.600098  3210.000000    2131   
1341  2025-05-02  3239.899902  3257.000000  3225.000000  3231.899902    2131   

               GDP  Interest_rates  Inflation_rate   DebtToGDP  ETF_Value  
1081  2.466163e+13        2.475952        2.475952  118.021316       0.25  
1101  2.466163e+13        2.475952        2.475952  118.021316       0.25  
1102  2.466163e+13        2.475952        2.475952  118.021316       0.25  
1103  2.466163e+13        2.475952        2.475952  118.021316       0.25  
1137  2.466163e+13        2.475952        2.475952  118.021316       0.25  
...            ...             ...             ...         ...        ...  
1337  2.466163e+13        2.475952        2.475952  118.021316       0.25  
1338  2.466163e+13        2.475952        2.475952  118.021316       0.25  
1339  2.466163e+13        2.475952        2.475952  118.021316       0.25  
1340  2.466163e+13        2.475952        2.475952  118.021316       0.25  
1341  2.466163e+13        2.475952        2.475952  118.021316       0.25  

[203 rows x 11 columns]

In [24]:
commodity_data

Date         Open         High          Low        Close  Volume  \
0     2020-01-02  1518.099976  1528.699951  1518.000000  1524.500000     214   
1     2020-01-03  1530.099976  1552.699951  1530.099976  1549.199951     107   
2     2020-01-06  1580.000000  1580.000000  1560.400024  1566.199951     416   
3     2020-01-07  1558.300049  1576.300049  1558.300049  1571.800049      47   
4     2020-01-08  1579.699951  1604.199951  1552.300049  1557.400024     236   
...          ...          ...          ...          ...          ...     ...   
1337  2025-04-28  3284.500000  3332.500000  3284.500000  3332.500000    1300   
1338  2025-04-29  3340.000000  3340.000000  3299.899902  3318.800049    1278   
1339  2025-04-30  3318.699951  3318.699951  3268.000000  3305.000000     207   
1340  2025-05-01  3272.899902  3275.000000  3198.600098  3210.000000    2131   
1341  2025-05-02  3239.899902  3257.000000  3225.000000  3231.899902    2131   

               GDP  Interest_rates  Inflation_rate   DebtToGDP  ETF_Value  
0     2.135410e+13        2.475952        2.475952  124.733177       0.25  
1     2.135410e+13        2.475952        2.475952  124.733177       0.25  
2     2.135410e+13        2.475952        2.475952  124.733177       0.25  
3     2.135410e+13        2.475952        2.475952  124.733177       0.25  
4     2.135410e+13        2.475952        2.475952  124.733177       0.25  
...            ...             ...             ...         ...        ...  
1337  2.466163e+13        2.475952        2.475952  118.021316       0.25  
1338  2.466163e+13        2.475952        2.475952  118.021316       0.25  
1339  2.466163e+13        2.475952        2.475952  118.021316       0.25  
1340  2.466163e+13        2.475952        2.475952  118.021316       0.25  
1341  2.466163e+13        2.475952        2.475952  118.021316       0.25  

[1342 rows x 11 columns]

###  Data Merging - Sentiments with Commodity Data

#### Merging Live and Histrical Sentiment data

In [25]:
live_sentiments_df = live_sentiments_df[['Date','Sentiment_score']]
historical_news_df = historical_news_df[['Date','Sentiment_score']]

In [26]:
# Convert 'Date' column to datetime format in both DataFrames
live_sentiments_df['Date'] = pd.to_datetime(live_sentiments_df['Date'])
historical_news_df['Date'] = pd.to_datetime(historical_news_df['Date'])

# Merge the DataFrames and sort by 'Date'
sentiment_df = pd.concat([live_sentiments_df, historical_news_df], ignore_index=True).sort_values(by='Date')

# Display the first few rows
print(sentiment_df.head())


            Date  Sentiment_score
5122  2000-02-15                0
10468 2000-02-17               -1
2759  2000-03-24               -1
3068  2000-04-27                1
9488  2000-05-04               -1


In [27]:
commodity_data['Date'] = pd.to_datetime(commodity_data['Date'])

In [28]:
commodity_data.tail()

Date         Open         High          Low        Close  Volume  \
1337 2025-04-28  3284.500000  3332.500000  3284.500000  3332.500000    1300   
1338 2025-04-29  3340.000000  3340.000000  3299.899902  3318.800049    1278   
1339 2025-04-30  3318.699951  3318.699951  3268.000000  3305.000000     207   
1340 2025-05-01  3272.899902  3275.000000  3198.600098  3210.000000    2131   
1341 2025-05-02  3239.899902  3257.000000  3225.000000  3231.899902    2131   

               GDP  Interest_rates  Inflation_rate   DebtToGDP  ETF_Value  
1337  2.466163e+13        2.475952        2.475952  118.021316       0.25  
1338  2.466163e+13        2.475952        2.475952  118.021316       0.25  
1339  2.466163e+13        2.475952        2.475952  118.021316       0.25  
1340  2.466163e+13        2.475952        2.475952  118.021316       0.25  
1341  2.466163e+13        2.475952        2.475952  118.021316       0.25

### 2.8 Merging Sentiment Data with Gold Data

In [29]:
commodity_data = pd.merge(commodity_data, sentiment_df[['Date', 'Sentiment_score']], on='Date', how='left')
commodity_data.head()

Date         Open         High          Low        Close  Volume  \
0 2020-01-02  1518.099976  1528.699951  1518.000000  1524.500000     214   
1 2020-01-03  1530.099976  1552.699951  1530.099976  1549.199951     107   
2 2020-01-06  1580.000000  1580.000000  1560.400024  1566.199951     416   
3 2020-01-07  1558.300049  1576.300049  1558.300049  1571.800049      47   
4 2020-01-08  1579.699951  1604.199951  1552.300049  1557.400024     236   

            GDP  Interest_rates  Inflation_rate   DebtToGDP  ETF_Value  \
0  2.135410e+13        2.475952        2.475952  124.733177       0.25   
1  2.135410e+13        2.475952        2.475952  124.733177       0.25   
2  2.135410e+13        2.475952        2.475952  124.733177       0.25   
3  2.135410e+13        2.475952        2.475952  124.733177       0.25   
4  2.135410e+13        2.475952        2.475952  124.733177       0.25   

   Sentiment_score  
0              NaN  
1              NaN  
2              NaN  
3              NaN  
4              NaN

In [30]:
commodity_data['Sentiment_score'].value_counts()

Sentiment_score
 0.0    526
-1.0    161
 1.0    159
Name: count, dtype: int64

### 2.9 Null Treatment post sentiment merge

In [31]:
#Interpolation – Estimate Sentiment Values Based on Nearby Data
#Good for gradual sentiment changes but may not capture sudden shifts.
commodity_data['Sentiment_score'] = sentiment_df['Sentiment_score'].interpolate(method='linear')  

commodity_data['Sentiment_score'].fillna(0, inplace=True)
# Print summary of missing values after imputation
print(commodity_data.isnull().sum())

Date               0
Open               0
High               0
Low                0
Close              0
Volume             0
GDP                0
Interest_rates     0
Inflation_rate     0
DebtToGDP          0
ETF_Value          0
Sentiment_score    0
dtype: int64


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_29016\1915627315.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  commodity_data['Sentiment_score'].fillna(0, inplace=True)


In [32]:
commodity_data['Sentiment_score'].value_counts()

Sentiment_score
 0    762
-1    717
 1    704
Name: count, dtype: int64

In [33]:
commodity_data

Date         Open         High          Low        Close  Volume  \
0    2020-01-02  1518.099976  1528.699951  1518.000000  1524.500000     214   
1    2020-01-03  1530.099976  1552.699951  1530.099976  1549.199951     107   
2    2020-01-06  1580.000000  1580.000000  1560.400024  1566.199951     416   
3    2020-01-07  1558.300049  1576.300049  1558.300049  1571.800049      47   
4    2020-01-08  1579.699951  1604.199951  1552.300049  1557.400024     236   
...         ...          ...          ...          ...          ...     ...   
2178 2025-05-01  3272.899902  3275.000000  3198.600098  3210.000000    2131   
2179 2025-05-01  3272.899902  3275.000000  3198.600098  3210.000000    2131   
2180 2025-05-01  3272.899902  3275.000000  3198.600098  3210.000000    2131   
2181 2025-05-01  3272.899902  3275.000000  3198.600098  3210.000000    2131   
2182 2025-05-02  3239.899902  3257.000000  3225.000000  3231.899902    2131   

               GDP  Interest_rates  Inflation_rate   DebtToGDP  ETF_Value  \
0     2.135410e+13        2.475952        2.475952  124.733177       0.25   
1     2.135410e+13        2.475952        2.475952  124.733177       0.25   
2     2.135410e+13        2.475952        2.475952  124.733177       0.25   
3     2.135410e+13        2.475952        2.475952  124.733177       0.25   
4     2.135410e+13        2.475952        2.475952  124.733177       0.25   
...            ...             ...             ...         ...        ...   
2178  2.466163e+13        2.475952        2.475952  118.021316       0.25   
2179  2.466163e+13        2.475952        2.475952  118.021316       0.25   
2180  2.466163e+13        2.475952        2.475952  118.021316       0.25   
2181  2.466163e+13        2.475952        2.475952  118.021316       0.25   
2182  2.466163e+13        2.475952        2.475952  118.021316       0.25   

      Sentiment_score  
0                   0  
1                   0  
2                   0  
3                   0  
4                  -1  
...               ...  
2178                1  
2179               -1  
2180               -1  
2181                1  
2182                1  

[2183 rows x 12 columns]

## Step 3 : Feature Engineering

### 4.1 Create Features 

In [34]:
import numpy as np

def feature_engineering(df, world_bank_data=None):
    # Lagged values and moving averages
    df['lag_1'] = df['Close'].shift(1)
    df['lag_7'] = df['Close'].shift(7)
    df['lag_30'] = df['Close'].shift(30)
    df['MA_7'] = df['Close'].rolling(window=7).mean()
    df['MA_30'] = df['Close'].rolling(window=30).mean()
    df['MA_90'] = df['Close'].rolling(window=90).mean()
    # Compute log returns fro GARCH
    df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))

    # Adding volatility features
    df['volatility_7'] = df['Close'].rolling(window=7).std()
    df['volatility_30'] = df['Close'].rolling(window=30).std()

    # #Gold prices may respond more to cumulative sentiment trends rather than individual daily sentiments. We apply a moving average to smooth fluctuations.
    # # Creating lagged sentiment features
    # df["Sentiment_Lag_1"] = df["Sentiment_score"].shift(1)
    # df["Sentiment_Lag_3"] = df["Sentiment_score"].shift(3)
    # df["Sentiment_Lag_7"] = df["Sentiment_score"].shift(7)
    # Compute rolling mean sentiment over 14-day and 30-day windows
    df["Sentiment_Rolling_14"] = df["Sentiment_score"].rolling(window=14, min_periods=1).mean()
    df["Sentiment_Rolling_30"] = df["Sentiment_score"].rolling(window=30, min_periods=1).mean()

    return df


### 4.2 Apply Feature engineering on data

In [35]:
commodity_data = feature_engineering(commodity_data)

In [36]:
feature_engineering(commodity_data)

Date         Open         High          Low        Close  Volume  \
0    2020-01-02  1518.099976  1528.699951  1518.000000  1524.500000     214   
1    2020-01-03  1530.099976  1552.699951  1530.099976  1549.199951     107   
2    2020-01-06  1580.000000  1580.000000  1560.400024  1566.199951     416   
3    2020-01-07  1558.300049  1576.300049  1558.300049  1571.800049      47   
4    2020-01-08  1579.699951  1604.199951  1552.300049  1557.400024     236   
...         ...          ...          ...          ...          ...     ...   
2178 2025-05-01  3272.899902  3275.000000  3198.600098  3210.000000    2131   
2179 2025-05-01  3272.899902  3275.000000  3198.600098  3210.000000    2131   
2180 2025-05-01  3272.899902  3275.000000  3198.600098  3210.000000    2131   
2181 2025-05-01  3272.899902  3275.000000  3198.600098  3210.000000    2131   
2182 2025-05-02  3239.899902  3257.000000  3225.000000  3231.899902    2131   

               GDP  Interest_rates  Inflation_rate   DebtToGDP  ...   lag_7  \
0     2.135410e+13        2.475952        2.475952  124.733177  ...     NaN   
1     2.135410e+13        2.475952        2.475952  124.733177  ...     NaN   
2     2.135410e+13        2.475952        2.475952  124.733177  ...     NaN   
3     2.135410e+13        2.475952        2.475952  124.733177  ...     NaN   
4     2.135410e+13        2.475952        2.475952  124.733177  ...     NaN   
...            ...             ...             ...         ...  ...     ...   
2178  2.466163e+13        2.475952        2.475952  118.021316  ...  3210.0   
2179  2.466163e+13        2.475952        2.475952  118.021316  ...  3210.0   
2180  2.466163e+13        2.475952        2.475952  118.021316  ...  3210.0   
2181  2.466163e+13        2.475952        2.475952  118.021316  ...  3210.0   
2182  2.466163e+13        2.475952        2.475952  118.021316  ...  3210.0   

      lag_30         MA_7        MA_30        MA_90  Log_Returns  \
0        NaN          NaN          NaN          NaN          NaN   
1        NaN          NaN          NaN          NaN     0.016072   
2        NaN          NaN          NaN          NaN     0.010914   
3        NaN          NaN          NaN          NaN     0.003569   
4        NaN          NaN          NaN          NaN    -0.009204   
...      ...          ...          ...          ...          ...   
2178  3210.0  3210.000000  3210.000000  3210.000000     0.000000   
2179  3210.0  3210.000000  3210.000000  3210.000000     0.000000   
2180  3210.0  3210.000000  3210.000000  3210.000000     0.000000   
2181  3210.0  3210.000000  3210.000000  3210.000000     0.000000   
2182  3210.0  3213.128557  3210.729997  3210.243332     0.006799   

      volatility_7  volatility_30  Sentiment_Rolling_14  Sentiment_Rolling_30  
0              NaN            NaN              0.000000              0.000000  
1              NaN            NaN              0.000000              0.000000  
2              NaN            NaN              0.000000              0.000000  
3              NaN            NaN              0.000000              0.000000  
4              NaN            NaN             -0.200000             -0.200000  
...            ...            ...                   ...                   ...  
2178      0.000000       0.000000             -0.071429             -0.133333  
2179      0.000000       0.000000             -0.214286             -0.133333  
2180      0.000000       0.000000             -0.214286             -0.200000  
2181      0.000000       0.000000             -0.142857             -0.200000  
2182      8.277385       3.998357             -0.071429             -0.133333  

[2183 rows x 23 columns]

In [37]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compute correlation of all features with 'Close'
correlation_with_close = commodity_data.corr()["Close"].sort_values(ascending=False)

# Convert to DataFrame
correlation_df = correlation_with_close.to_frame().rename(columns={"Close": "Correlation"})

correlation_df


Correlation
Close                    1.000000
Low                      0.999837
lag_1                    0.999690
High                     0.999544
MA_7                     0.999441
Open                     0.999199
lag_7                    0.997964
MA_30                    0.997546
lag_30                   0.992297
MA_90                    0.991184
Date                     0.887127
Inflation_rate           0.035435
Interest_rates           0.035435
GDP                      0.032082
Log_Returns             -0.001435
DebtToGDP               -0.009028
Sentiment_score         -0.024920
Volume                  -0.077508
Sentiment_Rolling_14    -0.094754
Sentiment_Rolling_30    -0.128050
volatility_7            -0.491768
volatility_30           -0.495205
ETF_Value                     NaN

### 4.3 Seting Index of data frame is as a Date column

In [38]:
commodity_data_df = commodity_data
commodity_data_df = commodity_data_df.drop_duplicates(subset='Date', keep='first')
commodity_data_df.set_index('Date', drop=True, inplace=True)
commodity_data_df = commodity_data_df.fillna(0.0)

In [39]:
commodity_data_df

Open         High          Low        Close  Volume  \
Date                                                                     
2020-01-02  1518.099976  1528.699951  1518.000000  1524.500000     214   
2020-01-03  1530.099976  1552.699951  1530.099976  1549.199951     107   
2020-01-06  1580.000000  1580.000000  1560.400024  1566.199951     416   
2020-01-07  1558.300049  1576.300049  1558.300049  1571.800049      47   
2020-01-08  1579.699951  1604.199951  1552.300049  1557.400024     236   
...                 ...          ...          ...          ...     ...   
2025-04-28  3284.500000  3332.500000  3284.500000  3332.500000    1300   
2025-04-29  3340.000000  3340.000000  3299.899902  3318.800049    1278   
2025-04-30  3318.699951  3318.699951  3268.000000  3305.000000     207   
2025-05-01  3272.899902  3275.000000  3198.600098  3210.000000    2131   
2025-05-02  3239.899902  3257.000000  3225.000000  3231.899902    2131   

                     GDP  Interest_rates  Inflation_rate   DebtToGDP  \
Date                                                                   
2020-01-02  2.135410e+13        2.475952        2.475952  124.733177   
2020-01-03  2.135410e+13        2.475952        2.475952  124.733177   
2020-01-06  2.135410e+13        2.475952        2.475952  124.733177   
2020-01-07  2.135410e+13        2.475952        2.475952  124.733177   
2020-01-08  2.135410e+13        2.475952        2.475952  124.733177   
...                  ...             ...             ...         ...   
2025-04-28  2.466163e+13        2.475952        2.475952  118.021316   
2025-04-29  2.466163e+13        2.475952        2.475952  118.021316   
2025-04-30  2.466163e+13        2.475952        2.475952  118.021316   
2025-05-01  2.466163e+13        2.475952        2.475952  118.021316   
2025-05-02  2.466163e+13        2.475952        2.475952  118.021316   

            ETF_Value  ...        lag_7       lag_30         MA_7  \
Date                   ...                                          
2020-01-02       0.25  ...     0.000000     0.000000     0.000000   
2020-01-03       0.25  ...     0.000000     0.000000     0.000000   
2020-01-06       0.25  ...     0.000000     0.000000     0.000000   
2020-01-07       0.25  ...     0.000000     0.000000     0.000000   
2020-01-08       0.25  ...     0.000000     0.000000     0.000000   
...               ...  ...          ...          ...          ...   
2025-04-28       0.25  ...  3282.399902  3282.399902  3289.557059   
2025-04-29       0.25  ...  3332.500000  3332.500000  3330.542864   
2025-04-30       0.25  ...  3318.800049  3318.800049  3316.828613   
2025-05-01       0.25  ...  3305.000000  3305.000000  3291.428571   
2025-05-02       0.25  ...  3210.000000  3210.000000  3213.128557   

                  MA_30        MA_90  Log_Returns  volatility_7  \
Date                                                              
2020-01-02     0.000000     0.000000     0.000000      0.000000   
2020-01-03     0.000000     0.000000     0.016072      0.000000   
2020-01-06     0.000000     0.000000     0.010914      0.000000   
2020-01-07     0.000000     0.000000     0.003569      0.000000   
2020-01-08     0.000000     0.000000    -0.009204      0.000000   
...                 ...          ...          ...           ...   
2025-04-28  3284.069906  3146.583290     0.015148     18.936057   
2025-04-29  3332.043335  3327.894436    -0.004119      5.178095   
2025-04-30  3318.340047  3319.103380    -0.004167      5.215928   
2025-05-01  3301.833333  3303.944444    -0.029166     35.906625   
2025-05-02  3210.729997  3210.243332     0.006799      8.277385   

            volatility_30  Sentiment_Rolling_14  Sentiment_Rolling_30  
Date                                                                   
2020-01-02       0.000000              0.000000              0.000000  
2020-01-03       0.000000              0.000000              0.000000  
2020-01-06       0.000000              0.000000              

### Step 5 Data Split and Scaling

In [40]:
import pandas as pd


# Convert index to datetime if not already
commodity_data_df.index = pd.to_datetime(commodity_data_df.index)

# Define 80-20% split based on row count
split_index = int(len(commodity_data_df) * 0.80) 
# split_date =  pd.to_datetime('2022-01-01')
split_date = commodity_data_df.index[split_index]  

# Train-test split
train_df = commodity_data_df.loc[:split_date]  
test_df = commodity_data_df.loc[split_date:]

print(f"Train size: {train_df.shape}, Test size: {test_df.shape}")


Train size: (1074, 22), Test size: (269, 22)


In [41]:
# Separate features and target

# Select features and target
X_train = train_df.drop(columns=[ 'Close' ])
y_train = train_df['Close']
X_test = test_df.drop(columns=[ 'Close'])
y_test = test_df['Close']
 

In [42]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Define scalers
scaler_minmax = MinMaxScaler()  # For XGBoost, Random Forest
scaler_standard = StandardScaler()  # For LSTM

# Apply MinMax Scaling (for tree-based models)
X_train_scaled_minmax = scaler_minmax.fit_transform(X_train)
X_test_scaled_minmax = scaler_minmax.transform(X_test)

# Apply Standard Scaling (for LSTM)
X_train_scaled_standard = scaler_standard.fit_transform(X_train)
X_test_scaled_standard = scaler_standard.transform(X_test)


# # Rescale y_train
# scaler_y = MinMaxScaler()
y_train_scaled = scaler_minmax.fit_transform(y_train.values.reshape(-1, 1)).ravel()
y_test_scaled = scaler_minmax.transform(y_test.values.reshape(-1, 1)).ravel()




### 8.1 Load pre trained model from File

In [43]:
import joblib
import pickle
import os
import pandas as pd
import numpy as np

# Define the model directory
model_dir = hybrid_models_dir.get(commodity.upper())

# Define the model directory
meta_dir = hybrid_meta_models_dir.get(commodity.upper())
hybridmodel_name = 'meta_model_'+commodity+'.pkl'
hybrid_meta_model_name = os.path.join(meta_dir, hybridmodel_name)
# Load the meta-model (used in stacked hybrid)
with open(hybrid_meta_model_name, 'rb') as f:
    meta_model = pickle.load(f)

# Function to load models
def load_saved_models(directory):
    model_files = [f for f in os.listdir(directory) if f.endswith('.pkl')]
    models = {}
    for file in model_files:
        model_path = os.path.join(directory, file)
        model_name = file.replace(".pkl", "")
        model = joblib.load(model_path)
        models[model_name] = model
    return models

# Load all saved models
loaded_models = load_saved_models(model_dir)


def predict_from_model(model_name, model):
    if "XGBOOST" in model_name.upper():
        model = model.fit(X_train_scaled_minmax, y_train_scaled)
        xgb_predictions = model.predict(X_test_scaled_minmax)
        xgb_prediction = scaler_minmax.inverse_transform(xgb_predictions.reshape(-1, 1)).ravel()
        y_test_org = scaler_minmax.inverse_transform(y_train_scaled.reshape(-1, 1)).ravel()
        return scaler_minmax.inverse_transform(xgb_predictions.reshape(-1, 1)).ravel()

    elif "RF" in model_name.upper():
        random_search = model.fit(X_train_scaled_minmax, y_train_scaled)
        y_pred_rf = random_search.predict(X_test_scaled_minmax)
        return  scaler_minmax.inverse_transform(y_pred_rf.reshape(-1, 1)).ravel()

    elif "LSTM" in model_name.upper():
        #  Select Sentiment + Microeconomic Features
        selected_features = [
            'Open', 'High', 'Low', 'Volume', 'lag_1', 'lag_7', 'lag_30',
            'MA_7', 'MA_30', 'MA_90', 'Log_Returns', 'volatility_7', 'volatility_30',
            'Sentiment_score', 'Sentiment_Rolling_14', 'Sentiment_Rolling_30'
        ]
        
        # Extract Features for Training & Testing
        X_train_selected = pd.DataFrame(X_train_scaled_standard, columns=X_train.columns)[selected_features].values
        X_test_selected = pd.DataFrame(X_test_scaled_standard, columns=X_test.columns)[selected_features].values
        
        #  Reshape Data for LSTM (3D: Samples, Time Steps, Features)
        look_back = 14  # Using the same look-back period
        
        def create_sequences(X, y, look_back):
            Xs, ys = [], []
            for i in range(len(X) - look_back):
                Xs.append(X[i:i + look_back])
                ys.append(y[i + look_back])
            return np.array(Xs), np.array(ys)
        
        X_train_seq, y_train_seq = create_sequences(X_train_selected, y_train_scaled, look_back)
        X_test_seq, y_test_seq = create_sequences(X_test_selected, y_test_scaled, look_back)
        #  Train Model
        model = model.fit(
            X_train_seq, y_train_seq,
            epochs=50, batch_size=32, 
            validation_data=(X_test_seq, y_test_seq),
            verbose=1
        )
        
        #  Predict on Test Set
        y_pred_lstm_scaled = model.predict(X_test_seq)
        
        #  Inverse Transform Predictions to Original Scale
        y_pred_lstm_original = scaler_minmax.inverse_transform(y_pred_lstm_scaled)
        return y_pred_lstm_original

    elif "ARIMA" in model_name.upper(): 
        # Select microeconomic factors from the scaled data
        # Select microeconomic factors from the scaled data
        exog_train = X_train_scaled_minmax[:, [X_train.columns.get_loc(col) for col in X_train.columns]]
        exog_test = X_test_scaled_minmax[:, [X_test.columns.get_loc(col) for col in X_test.columns]]
        model2 = model.get_forecast(steps=len(y_test), exog=exog_test)
        fc_model2 = model2.predicted_mean
        return fc_model2.values
        
    elif "GARCH" in model_name.upper():
        y_train_log_returns = np.log(y_train / y_train.shift(1)).dropna()
        y_test_log_returns = np.log(y_test / y_test.shift(1)).dropna()
        garch_forecast = model.forecast(horizon=len(y_test_log_returns))
        garch_forecast_series = garch_forecast.mean.iloc[-len(y_test_log_returns):].values.flatten()
        # Convert forecasted log returns back to price predictions
        y_test_prices = y_test.iloc[-len(garch_forecast_series):].values
        garch_predicted_prices = y_test_prices[0] * np.exp(np.cumsum(garch_forecast_series))
        return garch_predicted_prices
        
    elif "VAR" in model_name.upper():
        # Ensure data is stationary (apply differencing)
        train_var = train_df[['Close', 'Volume', 'MA_7', 'MA_30', 'MA_90', 'Log_Returns', 
                              'volatility_7', 'volatility_30', 'Sentiment_Rolling_14', 'Sentiment_Rolling_30']].diff().dropna()
        test_var = test_df[['Close', 'Volume', 'MA_7', 'MA_30', 'MA_90', 'Log_Returns', 
                            'volatility_7', 'volatility_30', 'Sentiment_Rolling_14', 'Sentiment_Rolling_30']].diff().dropna()
        
        # train_var = train_df[['Close', 'Volume', 'MA_7', 'MA_30']].diff().dropna()
        # test_var = test_df[['Close', 'Volume', 'MA_7', 'MA_30']].diff().dropna()
        train_exog = train_df[['GDP', 'Interest_rates', 'Inflation_rate', 'DebtToGDP', 'ETF_Value']].iloc[1:]
        test_exog = test_df[['GDP', 'Interest_rates', 'Inflation_rate', 'DebtToGDP', 'ETF_Value',]].iloc[1:]
        
        # train_exog = train_df[['GDP', 'Interest_rates', 'Inflation_rate', 'ETF_Value']].iloc[1:]  # Align with differencing
        # test_exog = test_df[['GDP', 'Interest_rates', 'Inflation_rate', 'ETF_Value']].iloc[1:]    # Align with differencing
        
        # Select optimal lag using BIC
        var_model = VAR(endog=train_var, exog=train_exog)
        lag_selection = var_model.select_order(maxlags=15)
        optimal_lag = lag_selection.bic
        print(f"Optimal lag order (BIC): {optimal_lag}")
        
        # Fit VAR model
        var_fitted = var_model.fit(optimal_lag)
        
        # Forecast
        forecast_steps = len(test_var)
        last_observations = train_var.values[-optimal_lag:]
        
        # Forecast with exogenous variables
        var_forecast = var_fitted.forecast(last_observations, steps=forecast_steps, exog_future=test_exog)
        
        # Convert forecasted differenced values back to original scale
        forecast_df_m4 = pd.DataFrame(var_forecast, columns=train_var.columns, index=test_var.index)
        forecast_df_m4 = train_df[['Close', 'Volume', 'MA_7', 'MA_30', 'MA_90', 'Log_Returns', 'volatility_7', 'volatility_30', 'Sentiment_Rolling_14', 'Sentiment_Rolling_30']].iloc[-1] + forecast_df_m4.cumsum()
        
        # Extract predicted Close prices
        predicted_prices = forecast_df_m4['Close']
        return predicted_prices
    else:
        print(f"Unknown model type for {model_name}. Skipping.")
        return None

predict_d= {}
for model_name, model in loaded_models.items():
    print(f"\nPredicting using model: {model_name}")
    prediction = predict_from_model(model_name, model)
    predict_d[model_name]=prediction
    if prediction is not None:
        print("Predicted values:", prediction)



Predicting using model: ARIMA_best-1
Predicted values: [2384.39422191 2372.68533084 2391.34418817 2385.58808217 2390.28245895
 2403.52568215 2386.13546829 2397.22575457 2403.95364303 2359.86244557
 2373.53831518 2373.38485646 2376.95963511 2379.28387726 2383.72889615
 2347.87707234 2367.74205492 2365.01991718 2364.46729841 2378.9367355
 2368.66414196 2369.22121086 2380.47401652 2397.49662048 2368.89469608
 2387.44458917 2404.02512979 2390.62820327 2411.41992074 2416.69170983
 2404.89352523 2387.90515397 2363.45197438 2376.10700151 2390.32031175
 2375.13644711 2380.0429145  2367.15897409 2386.12637363 2367.67579355
 2390.74655984 2393.16667098 2348.96713849 2368.21424681 2367.58364826
 2384.68608137 2355.26678911 2383.78665651 2363.99162515 2379.93359721
 2389.38373955 2360.85735019 2379.17923563 2367.48277186 2360.33237292
 2380.11490779 2375.58635289 2374.70739947 2371.81175142 2394.83271569
 2402.50433365 2375.83153511 2387.67068057 2393.56726677 2415.64120749
 2404.66098149 2410.98

C:\Users\Lenovo\myvenv\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Lenovo\myvenv\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


### 8.2 Load Pretrained Hybrid Model

In [44]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Get individual model forecasts
best1_forecast_trimmed, best2_forecast_trimmed = list(predict_d.values())

# Ensure both forecasts are the same length
min_length = min(len(best1_forecast_trimmed), len(best2_forecast_trimmed))

best1_forecast_trimmed = best1_forecast_trimmed[:min_length]
best2_forecast_trimmed = best2_forecast_trimmed[:min_length]

# Now combine
X_combined = np.column_stack((best1_forecast_trimmed, best2_forecast_trimmed))

# Normalize features
X_combined_scaled = scaler_minmax.fit_transform(X_combined)

# Predict using stacked model
hybrid_predictions_stacked = meta_model.predict(X_combined_scaled)


### 8.3 Predict Price with Stacked Hybrid Model

In [45]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

n = 1 # Number of future predictions

# Step 1: Ensure Predictions Are in NumPy Format
future_best1_values = np.array(best1_forecast_trimmed[-n:]).reshape(-1, 1)
future_best2_values = np.array(best2_forecast_trimmed[-n:]).reshape(-1, 1)

# Step 2: Prepare Feature Matrix for Future Predictions
X_combined_future = np.column_stack([future_best1_values, future_best2_values])

# Step 3: Fit MinMaxScaler on Training Data (if not already fitted)
scaler_minmax.fit(X_combined)  # Ensure the scaler is trained on past data
X_combined_future_scaled = scaler_minmax.transform(X_combined_future)

# Step 4: Predict Future Prices Using Hybrid Model
hybrid_predictions_future = meta_model.predict(X_combined_future_scaled)

# Step 5: Convert to 1D array
hybrid_predictions_future = hybrid_predictions_future.flatten()

# Step 6: Store Predictions in DataFrame
future_dates = pd.date_range(start=y_test.index[-1], periods=n + 1, freq='D')[1:]
future_price = pd.DataFrame({
    'Hybrid Forecast': hybrid_predictions_future
}, index=future_dates)

# Step 7: Display Predicted Future Prices
print("Predicted future values from the hybrid model:")
print(future_price)


Predicted future values from the hybrid model:
            Hybrid Forecast
2025-05-03      3275.417939


### 8.4 Let's verify Price with Weighted Hybrid Model

In [46]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt
min_length = min(len(best1_forecast_trimmed),len(best2_forecast_trimmed))
y_test_trimmed = y_test[:min_length].values.flatten()

# Calculate RMSE for both models
rmse_best1 = sqrt(mean_squared_error(y_test_trimmed, best1_forecast_trimmed))
rmse_best2 = sqrt(mean_squared_error(y_test_trimmed, best2_forecast_trimmed))

# Compute weights using inverse RMSE
weight_best1 = (1 / rmse_best1) / ((1 / rmse_best1) + (1 / rmse_best2))
weight_best2 = (1 / rmse_best2) / ((1 / rmse_best1) + (1 / rmse_best2))

# Weighted hybrid model predictions
hybrid_weighted_predictions = (weight_best1 * best1_forecast_trimmed) + (weight_best2 * best2_forecast_trimmed)



### 8.5 Predict Price with Weighted Hybrid Model

In [47]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression



# Step 1: Ensure Predictions Are in NumPy Format
future_best1_values = np.array(best1_forecast_trimmed[-n:]).reshape(-1, 1)
future_best2_values = np.array(best2_forecast_trimmed[-n:]).reshape(-1, 1)

# Step 2: Compute Hybrid Model Predictions

## 1. Simple Averaging Hybrid Model
hybrid_avg_future = (future_best1_values.flatten() + future_best2_values.flatten()) / 2

## 2. Weighted Hybrid Model (Based on RMSE Inverse Weights)
weight_best1 = (1 / rmse_best1) / ((1 / rmse_best1) + (1 / rmse_best2))
weight_best2 = (1 / rmse_best2) / ((1 / rmse_best1) + (1 / rmse_best2))
hybrid_weighted_future = (weight_best1 * future_best1_values.flatten()) + (weight_best2 * future_best2_values.flatten())

# Step 3: Store Predictions in DataFrame
future_dates = pd.date_range(start=y_test.index[-1], periods=n + 1, freq='D')[1:]
future_price = pd.DataFrame({
    'Hybrid_Weighted': hybrid_weighted_future,
}, index=future_dates)
print(future_price)

            Hybrid_Weighted
2025-05-03      3188.217338


### 8.6 Future Price Prediction and recommendation to Buy or Sell

### Predciting with Historical and Real time to check performance in real time

In [48]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

n = 10  # Number of future predictions
threshold = 100  # Decision threshold in USD

# Step 1: Ensure Predictions Are in NumPy Format
future_best1_values = np.array(best1_forecast_trimmed[-n:]).reshape(-1, 1)
future_best2_values = np.array(best2_forecast_trimmed[-n:]).reshape(-1, 1)

# Step 2: Prepare Feature Matrix for Future Predictions
X_combined_future = np.column_stack([future_best1_values, future_best2_values])

# Step 3: Scale and Predict
scaler_minmax.fit(X_combined)  # Use previously prepared training combined features
X_combined_future_scaled = scaler_minmax.transform(X_combined_future)

# Hybrid predictions
hybrid_predictions_weighted = (weight_best1 * future_best1_values.flatten()) + (weight_best2 * future_best2_values.flatten())
hybrid_predictions_stacked = meta_model.predict(X_combined_future_scaled).flatten()
hybrid_predictions_average = (future_best1_values.flatten() + future_best2_values.flatten()) / 2

# Step 4: Store Future Predictions
future_dates = pd.date_range(start=y_test.index[-1], periods=n + 1, freq='D')[1:]
future_price = pd.DataFrame({
    'Future Price Weighted': hybrid_predictions_weighted,
    'Future Price Stacked': hybrid_predictions_stacked,
    'Future Price Average': hybrid_predictions_average
}, index=future_dates)

# Step 5: Last Observed Price
last_actual_price = y_test.values[-1]

# Step 6: Recommendation Based on Weighted Hybrid
recommendation = []
for forecast in hybrid_predictions_weighted:
    price_diff = forecast - last_actual_price
    if price_diff > threshold:
        recommendation.append("BUY 📈")
    elif price_diff < -threshold:
        recommendation.append("SELL 📉")
    else:
        recommendation.append("HOLD ⚖️")

future_price['Recommendation'] = recommendation

# Display Predictions
print(f"Last Actual Observed Price: {last_actual_price:.2f}\n")
print("Predicted future values and investment recommendation:")
print(future_price)

# ======================================
# Evaluation Metrics on Known y_test[-n:]
# ======================================
actual_values = y_test[-n:].values.flatten()

def mape(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def accuracy(y_true, y_pred):
    return 100 - mape(y_true, y_pred)

# Custom metrics
def mape(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def accuracy(y_true, y_pred):
    return 100 - mape(y_true, y_pred)

def me(y_true, y_pred):
    return np.mean(y_pred - y_true)

# Store all evaluation metrics
metrics_summary = {
    "Model": ["Average Hybrid", "Weighted Hybrid", "Stacked Hybrid"],
    
    "ME": [
        me(actual_values, hybrid_predictions_average),
        me(actual_values, hybrid_predictions_weighted),
        me(actual_values, hybrid_predictions_stacked)
    ],
    "MSE": [
        mean_squared_error(actual_values, hybrid_predictions_average),
        mean_squared_error(actual_values, hybrid_predictions_weighted),
        mean_squared_error(actual_values, hybrid_predictions_stacked)
    ],
    "MAE": [
        mean_absolute_error(actual_values, hybrid_predictions_average),
        mean_absolute_error(actual_values, hybrid_predictions_weighted),
        mean_absolute_error(actual_values, hybrid_predictions_stacked)
    ],
    "RMSE": [
        np.sqrt(mean_squared_error(actual_values, hybrid_predictions_average)),
        np.sqrt(mean_squared_error(actual_values, hybrid_predictions_weighted)),
        np.sqrt(mean_squared_error(actual_values, hybrid_predictions_stacked))
    ],
    "R² Score": [
        r2_score(actual_values, hybrid_predictions_average),
        r2_score(actual_values, hybrid_predictions_weighted),
        r2_score(actual_values, hybrid_predictions_stacked)
    ],
    "MAPE (%)": [
        mape(actual_values, hybrid_predictions_average),
        mape(actual_values, hybrid_predictions_weighted),
        mape(actual_values, hybrid_predictions_stacked)
    ],
    "Accuracy (%)": [
        accuracy(actual_values, hybrid_predictions_average),
        accuracy(actual_values, hybrid_predictions_weighted),
        accuracy(actual_values, hybrid_predictions_stacked)
    ]
}

# Display as DataFrame
metrics_df = pd.DataFrame(metrics_summary)
print("\nEvaluation Metrics (on last known actual prices):")
print(metrics_df.round(4))

# Convert to DataFrame and show
metrics_df = pd.DataFrame(metrics_summary)
print("\nEvaluation Metrics (on last known actual prices):")
print(metrics_df.round(4))


Last Actual Observed Price: 3231.90

Predicted future values and investment recommendation:
            Future Price Weighted  Future Price Stacked  Future Price Average  \
2025-05-03            3315.612902           3372.923218           3068.424445   
2025-05-04            3378.113628           3419.167338           3098.323608   
2025-05-05            3269.305074           3335.998890           3014.043768   
2025-05-06            3261.959250           3331.853267           3026.145382   
2025-05-07            3268.969250           3336.058365           3017.612964   
2025-05-08            3237.886123           3313.826577           3012.025466   
2025-05-09            3273.763767           3340.234052           3027.513745   
2025-05-10            3256.682256           3327.291221           3015.658248   
2025-05-11            3208.437783           3289.806406           2970.917606   
2025-05-12            3188.217338           3275.417939           2968.177664   

           Recom

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# === Settings ===
n =10  # Number of future predictions (default: 5 days)2
threshold = 10  # Decision threshold for recommendation

# === Prepare Future Predictions ===
future_best1_values = np.array(best1_forecast_trimmed[-n:]).reshape(-1, 1)
future_best2_values = np.array(best2_forecast_trimmed[-n:]).reshape(-1, 1)

X_combined_future = np.column_stack([future_best1_values, future_best2_values])
scaler_minmax.fit(X_combined)  # Fit on previous features
X_combined_future_scaled = scaler_minmax.transform(X_combined_future)

# Stacked prediction
hybrid_predictions_future = meta_model.predict(X_combined_future_scaled).flatten()
# Weighted prediction
hybrid_weighted_future = (weight_best1 * future_best1_values.flatten()) + (weight_best2 * future_best2_values.flatten())
# Average prediction
hybrid_average_future = (future_best1_values.flatten() + future_best2_values.flatten()) / 2

# === Create Forecast DataFrame ===
future_dates = pd.date_range(start=y_test.index[-1], periods=n + 1, freq='D')[1:]
future_price = pd.DataFrame({
    'Future Price_stacked': hybrid_predictions_future,
}, index=future_dates)

# === Recommendation Logic ===
last_actual_price = y_test.values[-1]
recommendations = []
for forecast in hybrid_predictions_future:
    diff = forecast - last_actual_price
    if diff > threshold:
        recommendations.append("BUY 📈")
    elif diff < -threshold:
        recommendations.append("SELL 📉")
    else:
        recommendations.append("HOLD ⚖️")
future_price['Recommendation'] = recommendations

# === Trend & Signal Logic (Bonus) ===
trend = []
for i in range(len(hybrid_predictions_future)):
    if i >= 2:
        trend_window = hybrid_predictions_future[i-2:i+1]
        if np.all(np.diff(trend_window) > 0):
            trend.append("↗️ Uptrend Detected")
        elif np.all(np.diff(trend_window) < 0):
            trend.append("↘️ Downtrend Detected")
        else:
            trend.append("")
    else:
        trend.append("")
future_price['Signal'] = trend

# === Display Summary ===
print(f"Last Actual Observed Price: {last_actual_price:.2f}\n")
print(" Predicted Future Prices and Strategy:\n")
print(future_price)

# === Plot Forecasts ===
plt.figure(figsize=(12, 6))
plt.plot(future_price.index, future_price['Future Price_stacked'], label='Stacked Model', marker='o')

plt.axhline(last_actual_price, color='gray', linestyle=':', label=f'Last Actual Price ({last_actual_price:.2f})')
plt.fill_between(future_price.index, last_actual_price + threshold, last_actual_price - threshold,
                 color='yellow', alpha=0.1, label='HOLD Range (±$10)')
plt.title(" 10-Day Forecast: Stacked, Weighted & Average Models")
plt.xlabel("Date")
plt.ylabel("Predicted Price")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


Last Actual Observed Price: 3231.90

 Predicted Future Prices and Strategy:

            Future Price_stacked Recommendation                 Signal
2025-05-03           3372.923218          BUY 📈                       
2025-05-04           3419.167338          BUY 📈                       
2025-05-05           3335.998890          BUY 📈                       
2025-05-06           3331.853267          BUY 📈  ↘️ Downtrend Detected
2025-05-07           3336.058365          BUY 📈                       
2025-05-08           3313.826577          BUY 📈                       
2025-05-09           3340.234052          BUY 📈                       
2025-05-10           3327.291221          BUY 📈                       
2025-05-11           3289.806406          BUY 📈  ↘️ Downtrend Detected
2025-05-12           3275.417939          BUY 📈  ↘️ Downtrend Detected


## Predicting only with Real Time Data

In [50]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
n = days - 7
# Define scalers
scaler_minmax = MinMaxScaler()  # For XGBoost, Random Forest
scaler_standard = StandardScaler()  # For LSTM

# Apply MinMax Scaling (for tree-based models)
X_train_scaled_minmax = scaler_minmax.fit_transform(X_train)
X_test_scaled_minmax = scaler_minmax.transform(X_test)

# Apply Standard Scaling (for LSTM)
X_train_scaled_standard = scaler_standard.fit_transform(X_train)
X_test_scaled_standard = scaler_standard.transform(X_test)

# Select features and target
commodity_data_df = commodity_data_df.drop(columns=[ 'Close' ])
live_X_train = commodity_data_df[-n:]

# Apply existing scalers (already fitted during training)
X_live_scaled_minmax = scaler_minmax.transform(live_X_train)
X_live_scaled_standard = scaler_standard.transform(live_X_train)  # If needed for LSTM

scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))


In [51]:
import joblib
import pickle
import os
import pandas as pd
import numpy as np

# Define the model directory
model_dir = hybrid_models_dir.get(commodity.upper())

# Define the model directory
meta_dir = hybrid_meta_models_dir.get(commodity.upper())
hybridmodel_name = 'meta_model_'+commodity+'.pkl'
hybrid_meta_model_name = os.path.join(meta_dir, hybridmodel_name)
# Load the meta-model (used in stacked hybrid)
with open(hybrid_meta_model_name, 'rb') as f:
    meta_model = pickle.load(f)

# Function to load models
def load_saved_models(directory):
    model_files = [f for f in os.listdir(directory) if f.endswith('.pkl')]
    models = {}
    for file in model_files:
        model_path = os.path.join(directory, file)
        model_name = file.replace(".pkl", "")
        model = joblib.load(model_path)
        models[model_name] = model
    return models

# Load all saved models
loaded_models = load_saved_models(model_dir)


def predict_from_model(model_name, model):
    if "XGBOOST" in model_name.upper():
        # Use model directly without re-fitting
        xgb_predictions = model.predict(X_live_scaled_minmax)
        
        # Inverse transform predictions back to original scale
        xgb_prediction = scaler_y.inverse_transform(xgb_predictions.reshape(-1, 1)).ravel()
        return xgb_prediction

    elif "ARIMA" in model_name.upper(): 
        # Prepare exogenous variables from live data (same order as during training)
        exog_live = X_live_scaled_minmax[:, [X_train.columns.get_loc(col) for col in X_train.columns]]
        # Use pretrained ARIMA model to forecast
        steps = len(exog_live)
        model2 = model.get_forecast(steps=steps, exog=exog_live)
        fc_model2 = model2.predicted_mean
        
        return fc_model2.values

        print(f"Unknown model type for {model_name}. Skipping.")
        return None

predict_l= {}
for model_name, model in loaded_models.items():
    print(f"\nPredicting using model: {model_name}")
    prediction = predict_from_model(model_name, model)
    predict_l[model_name]=prediction
    if prediction is not None:
        print("Predicted values:", prediction)



Predicting using model: ARIMA_best-1
Predicted values: [2654.25285564 2664.52587223 2641.00916706 2607.22592581 2589.02159688
 2605.87048934 2643.69966243 2678.46434402 2698.71993396 2684.16485495
 2692.73535424 2738.17296762 2720.94657706 2764.68775747 2754.52734265
 2700.38734052 2736.38524742 2708.75498065 2734.49551288 2724.93007858
 2719.49618388 2679.06104146 2697.80039565]

Predicting using model: XGBoost_best-2
Predicted values: [2818.5195 2885.8914 2891.5923 2872.645  2805.0942 2774.4087 2775.0693
 2834.2036 2894.0352 2916.4917 2916.7754 2949.579  3014.778  3027.4666
 3090.5188 3034.797  2999.5037 3016.3481 3036.7908 3089.2505 3087.1562
 3044.7852 3004.7666]


C:\Users\Lenovo\myvenv\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Lenovo\myvenv\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [52]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Get individual model forecasts
best1_forecast_trimmed, best2_forecast_trimmed = list(predict_l.values())

# Ensure both forecasts are the same length
min_length = min(len(best1_forecast_trimmed), len(best2_forecast_trimmed))

best1_forecast_trimmed = best1_forecast_trimmed[:min_length]
best2_forecast_trimmed = best2_forecast_trimmed[:min_length]

# Now combine
X_combined = np.column_stack((best1_forecast_trimmed, best2_forecast_trimmed))

# Normalize features
X_combined_scaled = scaler_minmax.fit_transform(X_combined)

# Predict using stacked model
hybrid_predictions_stacked = meta_model.predict(X_combined_scaled)


In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
n = 10
# === Settings ===
# Number of future predictions (default: 5 days)2
threshold = 10  # Decision threshold for recommendation

# === Prepare Future Predictions ===
future_best1_values = np.array(best1_forecast_trimmed[-n:]).reshape(-1, 1)
future_best2_values = np.array(best2_forecast_trimmed[-n:]).reshape(-1, 1)

X_combined_future = np.column_stack([future_best1_values, future_best2_values])
scaler_minmax.fit(X_combined)  # Fit on previous features
X_combined_future_scaled = scaler_minmax.transform(X_combined_future)

# Stacked prediction
hybrid_predictions_future = meta_model.predict(X_combined_future_scaled).flatten()
# Weighted prediction
hybrid_weighted_future = (weight_best1 * future_best1_values.flatten()) + (weight_best2 * future_best2_values.flatten())
# Average prediction
hybrid_average_future = (future_best1_values.flatten() + future_best2_values.flatten()) / 2

# === Create Forecast DataFrame ===
future_dates = pd.date_range(start=y_test.index[-1], periods=n , freq='D')
future_price = pd.DataFrame({
    'Future Price_stacked': hybrid_predictions_future,
}, index=future_dates)

# === Recommendation Logic ===
last_actual_price = y_test.values[-1]
recommendations = []
for forecast in hybrid_predictions_future:
    diff = forecast - last_actual_price
    if diff > threshold:
        recommendations.append("BUY")
    elif diff < -threshold:
        recommendations.append("SELL")
    else:
        recommendations.append("HOLD")
future_price['Recommendation'] = recommendations

# === Trend & Signal Logic (Bonus) ===
trend = []
for i in range(len(hybrid_predictions_future)):
    if i >= 2:
        trend_window = hybrid_predictions_future[i-2:i+1]
        if np.all(np.diff(trend_window) > 0):
            trend.append("↗️ Uptrend Detected")
        elif np.all(np.diff(trend_window) < 0):
            trend.append("↘️ Downtrend Detected")
        else:
            trend.append("")
    else:
        trend.append("")
future_price['Signal'] = trend

# === Display Summary ===
print(f"Last Actual Observed Price: {last_actual_price:.2f}\n")
print(" Predicted Future Prices and Strategy:\n")
print(future_price)

# === Plot Forecasts ===
plt.figure(figsize=(12, 6))
plt.plot(future_price.index, future_price['Future Price_stacked'], label='Stacked Model', marker='o')

plt.axhline(last_actual_price, color='gray', linestyle=':', label=f'Last Actual Price ({last_actual_price:.2f})')
plt.fill_between(future_price.index, last_actual_price + threshold, last_actual_price - threshold,
                 color='yellow', alpha=0.1, label='HOLD Range (±$10)')
plt.title(f" {n}-Day Gold Forecast: Stacked Models")
plt.xlabel("Date")
plt.ylabel("Predicted Price")
plt.legend()
plt.grid(True)
plt.tight_layout()

# === Add Annotations with Boundary-Aware Positioning ===
ymin, ymax = plt.ylim()
text_offset = (ymax - ymin) * 0.03  # dynamic offset ~3% of range

for x, y, rec, sig in zip(future_price.index, 
                          future_price['Future Price_stacked'], 
                          future_price['Recommendation'], 
                          future_price['Signal']):
    # Calculate safe y-position for text (avoid going above ymax)
    y_text = y + text_offset
    if y_text > ymax:
        y_text = y - text_offset * 2  # put below if too high

    plt.text(x, y_text, rec, fontsize=9, ha='center', va='bottom')

    # Optional: show signal trend below the point
    if sig:
        plt.text(x, y - text_offset * 2, sig, fontsize=8, ha='center', va='top', color='blue')

# Optional: Rotate x-axis labels
plt.xticks(rotation=45)

plt.show()


Last Actual Observed Price: 3231.90

 Predicted Future Prices and Strategy:

            Future Price_stacked Recommendation                 Signal
2025-05-02           3255.823302            BUY                       
2025-05-03           3417.682097            BUY                       
2025-05-04           3258.693230            BUY                       
2025-05-05           3175.751330           SELL  ↘️ Downtrend Detected
2025-05-06           3212.696698           SELL                       
2025-05-07           3272.510200            BUY    ↗️ Uptrend Detected
2025-05-08           3406.896407            BUY    ↗️ Uptrend Detected
2025-05-09           3400.062147            BUY                       
2025-05-10           3279.351338            BUY  ↘️ Downtrend Detected
2025-05-11           3179.726804           SELL  ↘️ Downtrend Detected
